In [76]:
import pandas as pd
import json
import os
from tqdm import tqdm
import pickle

gather a list of failed video ids
 - api fail
 - no video detail
create two df pickled files holding all successful video details and comments
 - videoID as keys
 - some videos disabled comments so n_video in video details will be more than n_video in comment file

In [3]:
# read in api failure list 
fp = '../data/raw/api_data/failed_lookups.txt'

with open(fp) as f:
    failed_dict = json.load(f)
    
failed_dict.keys()

dict_keys(['failed'])

In [60]:
failed_ids = set([lst[0] for lst in failed_dict['failed']]) #273

In [94]:
with open('../data/processed/failed_ids.pkl', 'wb') as f:
    pickle.dump(failed_ids, f)

In [9]:
# read in test_batch 
test_batch = '../data/processed/test_batch.txt'

with open(test_batch) as tb:
    test_files = json.load(tb)

test_files.keys()

dict_keys(['HTft', 'VX3C', 'j2Er', '3HOO', '6nfW', 'NX6e', 'ljAU', 'zmk6', 'qX7a', '3Z4A', 'kugn', '2D9o', 'Tvnd', 'A9Jp', 'f3CR'])

In [119]:
def get_videoDetails(data, report = False):
    '''return a dataframe of all video details features from Youtube API and a set of failed id'''

    # for every yt8M id, check if videoDetail is available
    # if not, check if it's in failure list, if not, add to failure list
    # if it is, append to buffer list 
    
    buffer = []
    failed_ids = set()
    for idx in data.keys():
        if data[idx]['details'] == 'Invalid lookup.':
            failed_ids.add(idx) 
        elif len(data[idx]['details']['items']) == 0:
            failed_ids.add(idx)
        else:
            data[idx]['details']['items'][0]['yt8M_id'] = idx
            buffer.append(data[idx]['details']['items'][0])
    
    # json_normalize buffer list into videoDetails df
    videoDets_df = pd.json_normalize(buffer, errors='ignore')
    
    # print stats for batch
    if report:
        print(f'n of videos successful: {videoDets_df.shape[0]}')
        print(f'n of videos failed: {len(failed_ids)}')
    
    return videoDets_df, failed_ids
    
def get_videoComments(data, report=False):
    '''return a dataframe of all comments features from Youtube API'''
    
    no_comment_count = 0
    buffer = []
    for idx in data.keys():
        if data[idx]['comments'] == 'Invalid lookup':
            no_comment_count += 1
        elif len(data[idx]['comments']['items']) == 0:
            no_comment_count += 1
        else:
            data[idx]['comments']['items'][0]['yt8M_id'] = idx
            buffer.append(data[idx]['comments']['items'][0])
    
    # json_normalize buffer list into videoDetails df
    videoComs_df = pd.json_normalize(buffer, errors='ignore')
    
    # print stats for batch
    if report:
        print(f'{no_comment_count = }')

    return videoComs_df

In [124]:
test_df, _ = get_videoDetails(test_files, report=True)
test_df.head()

n of videos successful: 15
n of videos failed: 0


,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,status.embeddable,status.publicStatsViewable,status.madeForKids,statistics.viewCount,statistics.likeCount,statistics.favoriteCount,statistics.commentCount,player.embedHtml,topicDetails.topicCategories,recordingDetails.recordingDate
0,youtube#video,qw6REl1mcyQq7UoW1-Dkm1yCrOY,130S1TN3P0M,HTft,2014-11-05T07:49:51Z,UCh_KdtEAc7qwyeXyVuuYPUA,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,https://i.ytimg.com/vi/130S1TN3P0M/default.jpg,120,...,True,True,False,2415,19,0,3,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,NaN
1,youtube#video,hBay_xFV_egFO5I_5lVHS8xcSi4,iHpgw8WN6Bg,VX3C,2011-11-01T06:43:23Z,UCgBA-PogWxh44p-lKZW9EVA,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...",https://i.ytimg.com/vi/iHpgw8WN6Bg/default.jpg,120,...,True,True,False,5582,27,0,1,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2011-10-31T00:00:00Z
2,youtube#video,R0zGKh5qNMofm0IBhZHvobnWkqM,XrX9NBMUOzw,j2Er,2006-08-05T18:57:44Z,UC4ONXuixSMf5o9O67A4gzhg,Two Giraffe Calves born at the San Francisco Zoo,Follow the zoo: \r\nhttp://twitter.com/sfzoo\r...,https://i.ytimg.com/vi/XrX9NBMUOzw/default.jpg,120,...,True,True,False,289627,245,0,59,"<iframe width=""480"" height=""360"" src=""//www.yo...",[https://en.wikipedia.org/wiki/Lifestyle_(soci...,2006-05-09T00:00:00Z
3,youtube#video,XY93dMczSpUYyP18PTAnGLAW7Ws,qwL5HKti-dw,3HOO,2009-12-03T14:46:09Z,UCWc9ziDlznO7v7cjzuT-Pqw,Fauna salbatica bucovineana in fotografi - Buk...,#WildBucovina Natura si fauna sălbatică din Bu...,https://i.ytimg.com/vi/qwL5HKti-dw/default.jpg,120,...,True,True,False,4456,23,0,5,"<iframe width=""480"" height=""360"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN
4,youtube#video,NcYca-t2DZKiXg5HhMYTuQeGBoI,wPJy9D_kzVU,6nfW,2010-07-13T06:23:18Z,UCtCCW9be1rUw_ig4UWsOSNg,Wyoming Archery Mule Deer Hunt With Joe Thomas...,,https://i.ytimg.com/vi/wPJy9D_kzVU/default.jpg,120,...,True,True,False,1338,2,0,0,"<iframe width=""480"" height=""270"" src=""//www.yo...","[https://en.wikipedia.org/wiki/Hobby, https://...",NaN


In [125]:
test_df = get_videoComments(test_files, report=True)
test_df.head()

no_comment_count = 4


,kind,etag,id,yt8M_id,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.videoId,snippet.topLevelComment.snippet.textDisplay,...,snippet.topLevelComment.snippet.authorChannelUrl,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic
0,youtube#commentThread,EJSyRFx1dhqppMPt15RFmeUpnnM,UgjMGXaQw47fE3gCoAEC,HTft,130S1TN3P0M,youtube#comment,H5M32lmblp9WMe-eHLfBPKr_GrE,UgjMGXaQw47fE3gCoAEC,130S1TN3P0M,2:56 jeśli się nie mylę to pardwa:) Występuje ...,...,http://www.youtube.com/channel/UCJrxiBRv5IHv1U...,UCJrxiBRv5IHv1UJfKULEW7Q,True,none,0,2017-04-26T09:48:13Z,2017-04-26T09:48:13Z,True,2,True
1,youtube#commentThread,Oh-NTWh_vFipTc-3BKBTVz0LTEs,UgxLCQCLsca5zhKol2h4AaABAg,VX3C,iHpgw8WN6Bg,youtube#comment,cTsnqv7CcJvgJ0pznTYOoEt7vwY,UgxLCQCLsca5zhKol2h4AaABAg,iHpgw8WN6Bg,<3<3<3<3<3<3\nTrop ADORABLES!,...,http://www.youtube.com/channel/UCkHRUDRkrJPUXQ...,UCkHRUDRkrJPUXQfar1hChOg,True,none,0,2012-03-12T14:27:16Z,2012-03-12T14:27:16Z,True,0,True
2,youtube#commentThread,KHGlSgVVApoRzxY9_nMqbYTXWtE,UgySkF4KV4tbia-wGAF4AaABAg,j2Er,XrX9NBMUOzw,youtube#comment,pMET6ORoowLzZavBMrKFS1dpZPg,UgySkF4KV4tbia-wGAF4AaABAg,XrX9NBMUOzw,I loved how momma tapped at the calf to get th...,...,http://www.youtube.com/channel/UCc3BnE0KTjJzf6...,UCc3BnE0KTjJzf621gdN8L-A,True,none,0,2008-02-18T05:44:42Z,2008-02-18T05:44:42Z,True,0,True
3,youtube#commentThread,sXQJ7lTCShzrGvoqvf-p7OkJKA4,UgyccrM8fmN9z7gpArF4AaABAg,3HOO,qwL5HKti-dw,youtube#comment,WO9MkbKm5F3-XdRnDzJ3tnVPeDQ,UgyccrM8fmN9z7gpArF4AaABAg,qwL5HKti-dw,"Dacă ți-a plăcut, dă un ""LIKE"" și un ""SHARE"", ...",...,http://www.youtube.com/channel/UCWc9ziDlznO7v7...,UCWc9ziDlznO7v7cjzuT-Pqw,True,none,0,2022-07-08T12:56:11Z,2022-07-08T12:56:11Z,True,0,True
4,youtube#commentThread,Bm7jY1bz8gU1S1tt2BVeF26uE1Y,UgwQDLyNGIReuoaqvS94AaABAg,NX6e,S-MjHWCF_Cw,youtube#comment,pwQZ8vjRu669O7VLPjr4HvACbQE,UgwQDLyNGIReuoaqvS94AaABAg,S-MjHWCF_Cw,One day I'll decide to harvest!!! Thanks Rober...,...,http://www.youtube.com/channel/UCu_YcCDBu_UCmX...,UCu_YcCDBu_UCmXf2nmpw3vg,True,none,0,2009-11-24T12:18:59Z,2009-11-24T12:18:59Z,True,0,True


In [129]:
# read in batch1
batch_1 = '../data/raw/api_data/batch_1.txt'

with open(batch_1) as b1:
    b1_videos = json.load(b1)

In [130]:
# get videoDetails and Comments
vDetails_df, _ = get_videoDetails(b1_videos, report=True)
vDetails_df.head()

n of videos successful: 393
n of videos failed: 8


,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,recordingDetails.locationDescription,recordingDetails.location.latitude,recordingDetails.location.longitude,recordingDetails.location.altitude,snippet.defaultLanguage,localizations.en-US.title,localizations.en-US.description,contentDetails.regionRestriction.blocked,contentDetails.contentRating.ytRating,contentDetails.regionRestriction.allowed
0,youtube#video,qw6REl1mcyQq7UoW1-Dkm1yCrOY,130S1TN3P0M,HTft,2014-11-05T07:49:51Z,UCh_KdtEAc7qwyeXyVuuYPUA,Obserwator 22: Jesień na tamie / Autumn on a ...,To już pożegnanie z tamą bobrową. Jesień zawit...,https://i.ytimg.com/vi/130S1TN3P0M/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,youtube#video,hBay_xFV_egFO5I_5lVHS8xcSi4,iHpgw8WN6Bg,VX3C,2011-11-01T06:43:23Z,UCgBA-PogWxh44p-lKZW9EVA,bébés ocelots et otocyons au Domaine des Fauves,"les bébés ocelots âgés de 7 semaines, arrivée ...",https://i.ytimg.com/vi/iHpgw8WN6Bg/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube#video,R0zGKh5qNMofm0IBhZHvobnWkqM,XrX9NBMUOzw,j2Er,2006-08-05T18:57:44Z,UC4ONXuixSMf5o9O67A4gzhg,Two Giraffe Calves born at the San Francisco Zoo,Follow the zoo: \r\nhttp://twitter.com/sfzoo\r...,https://i.ytimg.com/vi/XrX9NBMUOzw/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,youtube#video,XY93dMczSpUYyP18PTAnGLAW7Ws,qwL5HKti-dw,3HOO,2009-12-03T14:46:09Z,UCWc9ziDlznO7v7cjzuT-Pqw,Fauna salbatica bucovineana in fotografi - Buk...,#WildBucovina Natura si fauna sălbatică din Bu...,https://i.ytimg.com/vi/qwL5HKti-dw/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,youtube#video,NcYca-t2DZKiXg5HhMYTuQeGBoI,wPJy9D_kzVU,6nfW,2010-07-13T06:23:18Z,UCtCCW9be1rUw_ig4UWsOSNg,Wyoming Archery Mule Deer Hunt With Joe Thomas...,,https://i.ytimg.com/vi/wPJy9D_kzVU/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
vComments_df = get_videoComments(b1_videos, report=True)
vComments_df.head()

no_comment_count = 106


,kind,etag,id,yt8M_id,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.videoId,snippet.topLevelComment.snippet.textDisplay,...,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,snippet.topLevelComment.snippet.moderationStatus
0,youtube#commentThread,EJSyRFx1dhqppMPt15RFmeUpnnM,UgjMGXaQw47fE3gCoAEC,HTft,130S1TN3P0M,youtube#comment,H5M32lmblp9WMe-eHLfBPKr_GrE,UgjMGXaQw47fE3gCoAEC,130S1TN3P0M,2:56 jeśli się nie mylę to pardwa:) Występuje ...,...,UCJrxiBRv5IHv1UJfKULEW7Q,True,none,0,2017-04-26T09:48:13Z,2017-04-26T09:48:13Z,True,2,True,NaN
1,youtube#commentThread,Oh-NTWh_vFipTc-3BKBTVz0LTEs,UgxLCQCLsca5zhKol2h4AaABAg,VX3C,iHpgw8WN6Bg,youtube#comment,cTsnqv7CcJvgJ0pznTYOoEt7vwY,UgxLCQCLsca5zhKol2h4AaABAg,iHpgw8WN6Bg,<3<3<3<3<3<3\nTrop ADORABLES!,...,UCkHRUDRkrJPUXQfar1hChOg,True,none,0,2012-03-12T14:27:16Z,2012-03-12T14:27:16Z,True,0,True,NaN
2,youtube#commentThread,KHGlSgVVApoRzxY9_nMqbYTXWtE,UgySkF4KV4tbia-wGAF4AaABAg,j2Er,XrX9NBMUOzw,youtube#comment,pMET6ORoowLzZavBMrKFS1dpZPg,UgySkF4KV4tbia-wGAF4AaABAg,XrX9NBMUOzw,I loved how momma tapped at the calf to get th...,...,UCc3BnE0KTjJzf621gdN8L-A,True,none,0,2008-02-18T05:44:42Z,2008-02-18T05:44:42Z,True,0,True,NaN
3,youtube#commentThread,sXQJ7lTCShzrGvoqvf-p7OkJKA4,UgyccrM8fmN9z7gpArF4AaABAg,3HOO,qwL5HKti-dw,youtube#comment,WO9MkbKm5F3-XdRnDzJ3tnVPeDQ,UgyccrM8fmN9z7gpArF4AaABAg,qwL5HKti-dw,"Dacă ți-a plăcut, dă un ""LIKE"" și un ""SHARE"", ...",...,UCWc9ziDlznO7v7cjzuT-Pqw,True,none,0,2022-07-08T12:56:11Z,2022-07-08T12:56:11Z,True,0,True,NaN
4,youtube#commentThread,Bm7jY1bz8gU1S1tt2BVeF26uE1Y,UgwQDLyNGIReuoaqvS94AaABAg,NX6e,S-MjHWCF_Cw,youtube#comment,pwQZ8vjRu669O7VLPjr4HvACbQE,UgwQDLyNGIReuoaqvS94AaABAg,S-MjHWCF_Cw,One day I'll decide to harvest!!! Thanks Rober...,...,UCu_YcCDBu_UCmXf2nmpw3vg,True,none,0,2009-11-24T12:18:59Z,2009-11-24T12:18:59Z,True,0,True,NaN


In [83]:
# with open('../data/processed/batch1_videoDetails.pkl', 'wb') as f:
#     pickle.dump(vDetails_df, f)
    
# with open('../data/processed/batch1_videoComments.pkl', 'wb') as f:
#     pickle.dump(vComments_df, f)    

In [132]:
# trial code for reading in all data within directory
input_dir = '../data/raw/api_data'
all_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.startswith('batch')]
all_files

['../data/raw/api_data/batch_2.txt',
 '../data/raw/api_data/batch_3.txt',
 '../data/raw/api_data/batch_1.txt',
 '../data/raw/api_data/batch_4.txt',
 '../data/raw/api_data/batch_5.txt',
 '../data/raw/api_data/batch_7.txt',
 '../data/raw/api_data/batch_6.txt',
 '../data/raw/api_data/batch_11.txt',
 '../data/raw/api_data/batch_8.txt',
 '../data/raw/api_data/batch_9.txt',
 '../data/raw/api_data/batch_10.txt']

In [133]:
sample_files = all_files[:9]
sample_files

['../data/raw/api_data/batch_2.txt',
 '../data/raw/api_data/batch_3.txt',
 '../data/raw/api_data/batch_1.txt',
 '../data/raw/api_data/batch_4.txt',
 '../data/raw/api_data/batch_5.txt',
 '../data/raw/api_data/batch_7.txt',
 '../data/raw/api_data/batch_6.txt',
 '../data/raw/api_data/batch_11.txt',
 '../data/raw/api_data/batch_8.txt']

In [134]:
failure_ids = set()
vDets_buffer = []
vComs_buffer = []

for f in tqdm(sample_files):
    with open(f) as f:
        json_f = json.load(f)
    
    vDets_df, fail_ids = get_videoDetails(json_f, report=True)
    vDets_buffer.append(vDets_df)
    failed_ids.update(fail_ids)
        
    vComs_df = get_videoComments(json_f, report=True)
    vComs_buffer.append(vComs_df)
        
    
vDets = pd.concat(vDets_buffer,ignore_index=True)
vComs = pd.concat(vComs_buffer,ignore_index=True)    

 33%|███████████████                              | 3/9 [00:00<00:00, 25.96it/s]

n of videos successful: 396
n of videos failed: 4
no_comment_count = 400
n of videos successful: 394
n of videos failed: 6
no_comment_count = 400
n of videos successful: 393
n of videos failed: 8
no_comment_count = 106
n of videos successful: 394
n of videos failed: 6
no_comment_count = 400
n of videos successful: 392
n of videos failed: 8
no_comment_count = 400


100%|█████████████████████████████████████████████| 9/9 [00:00<00:00, 29.52it/s]


n of videos successful: 393
n of videos failed: 7
no_comment_count = 400
n of videos successful: 392
n of videos failed: 8
no_comment_count = 400
n of videos successful: 233
n of videos failed: 9
no_comment_count = 242
n of videos successful: 394
n of videos failed: 6
no_comment_count = 400


In [135]:
vDets.sample(15)

,kind,etag,id,yt8M_id,snippet.publishedAt,snippet.channelId,snippet.title,snippet.description,snippet.thumbnails.default.url,snippet.thumbnails.default.width,...,localizations.no.title,localizations.no.description,localizations.af.title,localizations.af.description,localizations.zu.title,localizations.zu.description,localizations.zh-Hans.title,localizations.zh-Hans.description,localizations.pt.title,localizations.pt.description
202,youtube#video,Wf_Ir4uzbNWJ2Q1JLkjrTwfgHe0,CRJxTn2c2q8,QmBI,2010-12-20T05:16:14Z,UCiaJyoMlrVy5x8VJl7ri3OA,Wack-Um with BoonerBuilder Invisible Food Plo...,,https://i.ytimg.com/vi/CRJxTn2c2q8/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2707,youtube#video,rRggJaCQxDsuVqU_PmtjvaITznI,rx3fR5WycX4,R6im,2013-10-23T05:01:53Z,UC3EV8iEXRkndqgWAbHjSCcg,"618RackRecruits: ""A long night""",Hunter Zeigler self-films a takedown of a 145 ...,https://i.ytimg.com/vi/rx3fR5WycX4/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553,youtube#video,44Zq31qrcO6W85J5fBIc3U1apiQ,1kw2M15zD_I,zWrh,2010-02-28T11:00:57Z,UC4xHqB2hRJHCPn-ZxMFt6cQ,Les animaux d'Afrique de A à Z,Les animaux d'Afrique de A à Z\r\n\r\nAutruche...,https://i.ytimg.com/vi/1kw2M15zD_I/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2790,youtube#video,nb1TGHlWlMaRrY5StBWan5KQr8M,zf2vCxLJAGk,UriR,2012-06-02T19:51:23Z,UCRfK_rNYUSYecRztV3YNFyA,деревянные поделки на даче видео,http://samodelpshelovod.ru/ деревянные поделки...,https://i.ytimg.com/vi/zf2vCxLJAGk/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,youtube#video,iBRYVPAIEjQ3JagfzAX8A44FC6s,CQdpx4F88-w,5sLi,2008-10-17T08:43:59Z,UCqPai4_Jn0WA_LRLx_1i6rA,Pumba Game Reserve,A presentation on the luxury you can expect to...,https://i.ytimg.com/vi/CQdpx4F88-w/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3080,youtube#video,FkR3KsdNs6y9Xuntemith800TOY,Zwa6Kwhny_Q,skuq,2007-07-09T04:06:34Z,UCEEIeRbcKlMF_e0Ts9ucIHg,Playing with Meerkats,Join Jeff the Zoo Guy as he brings you close u...,https://i.ytimg.com/vi/Zwa6Kwhny_Q/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2453,youtube#video,ujrGdToJwa9BrFpXDZKgU5sbG8c,GZJ-VB1SMrs,Lczj,2010-08-08T16:48:12Z,UCOyOg_gDMzdVvRrC0UfnbQA,Swamp Girl Adventures Reedy Creek Environmenta...,Swamp Girl and the crew visit the Osceola sch...,https://i.ytimg.com/vi/GZJ-VB1SMrs/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2167,youtube#video,LL7n8Ulwu-DAx4V5iYlIBfRgQQQ,cRD8U-Zm7hQ,tAub,2008-09-30T21:16:24Z,UCbzHmHqNtIYeFWdYy3QLIsw,Kenya Travel Doc - Part 06,The Republic of Kenya is a country in East Afr...,https://i.ytimg.com/vi/cRD8U-Zm7hQ/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1737,youtube#video,L9p2uHJzx92fw3dzyNO7VQ6Yjc8,62Q39gzrd-k,7Z3T,2010-10-29T23:29:45Z,UCQqsJFVfWTOiqVgpznt7n0g,watching two pope and young caribou get away,Hunting caribou on the arctic slope of Alaska ...,https://i.ytimg.com/vi/62Q39gzrd-k/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938,youtube#video,r5GNLbYph2NutsE2oNjD64UFTbE,Un3miVdN644,6Ktp,2011-06-23T01:26:37Z,UCvnRULs9LwKZvfJ67DVJt7g,panthera leo playtime,http://www.saveoursiberians.org Tonka and Shan...,https://i.ytimg.com/vi/Un3miVdN644/default.jpg,120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
vComs.sample(10)

,kind,etag,id,yt8M_id,snippet.videoId,snippet.topLevelComment.kind,snippet.topLevelComment.etag,snippet.topLevelComment.id,snippet.topLevelComment.snippet.videoId,snippet.topLevelComment.snippet.textDisplay,...,snippet.topLevelComment.snippet.authorChannelId.value,snippet.topLevelComment.snippet.canRate,snippet.topLevelComment.snippet.viewerRating,snippet.topLevelComment.snippet.likeCount,snippet.topLevelComment.snippet.publishedAt,snippet.topLevelComment.snippet.updatedAt,snippet.canReply,snippet.totalReplyCount,snippet.isPublic,snippet.topLevelComment.snippet.moderationStatus
64,youtube#commentThread,1oyiyDbpsgPNKNAjF8Fpqo7_dLM,UgyQc0x_eplLEFib-nN4AaABAg,geO1,51ZHs59Umj8,youtube#comment,jJ4MmQ1IRovmXCNYqwxf29ZL1BA,UgyQc0x_eplLEFib-nN4AaABAg,51ZHs59Umj8,"Lovely big eyes, nice found and good capture.",...,UCPBc41nPwx11H9P2peHXxJA,True,none,0,2008-04-10T12:50:39Z,2008-04-10T12:50:39Z,True,0,True,NaN
284,youtube#commentThread,-YXcgBFbIGXoyPSB7i2VRzeGVos,UgxBLWHcGTEMgtb_TL94AaABAg,m8mX,fuEygFPyR2Y,youtube#comment,t10Rbb2xDhgZYlHGFfMk95mvr8A,UgxBLWHcGTEMgtb_TL94AaABAg,fuEygFPyR2Y,Bet u wish it was deer season :-) :-) :-),...,UCziBihqPzZenz5RB0_UvuyA,True,none,0,2007-10-07T23:56:49Z,2007-10-07T23:56:49Z,True,0,True,NaN
269,youtube#commentThread,0KCRNN5Rw4RAIqnRaCBvkJby4fE,Ugx807sm0ChKG9T7ROh4AaABAg,Vl4B,rgBpmNJrQCY,youtube#comment,6HFPGLkhL5H2lN_iZRIaVMq2aGw,Ugx807sm0ChKG9T7ROh4AaABAg,rgBpmNJrQCY,泳ぐライオンいいね昔か水嫌がってたのに,...,UCyPXUiQwpUPsZwyLlM-4_EA,True,none,0,2020-12-19T04:32:11Z,2020-12-19T04:32:11Z,True,1,True,NaN
268,youtube#commentThread,YCj6PKmk_Lbc5iLMjLvJKVX83N0,UgxuwhOaXugKdMLg4YV4AaABAg,t2D0,B5bOQRLiIxo,youtube#comment,Cbg5uUG5A3OqkkVhHyRkU4uawmQ,UgxuwhOaXugKdMLg4YV4AaABAg,B5bOQRLiIxo,"Awesome buck, and some real good footage there...",...,UCZ0sQ4fM3Ei55-qEPxs-Q5A,True,none,0,2008-02-09T09:09:20Z,2008-02-09T09:09:20Z,True,0,True,NaN
23,youtube#commentThread,yDDUTKuOF7g_h1zz0dMOf-piVf8,UgzFLBEM0Hqk2PawqUl4AaABAg,xlED,Q9GIhJ6nzOg,youtube#comment,7UpXtJ3Zniw-ZR1KjzXAz1hzMWw,UgzFLBEM0Hqk2PawqUl4AaABAg,Q9GIhJ6nzOg,Why do I find pleasure in watching videos like...,...,UCmlp-saiIosd_Br_3XTXOPw,True,none,10,2013-06-20T20:37:12Z,2013-06-20T20:37:12Z,True,1,True,NaN
27,youtube#commentThread,ujw9cu8PDAf2ACK7IeQPhEgdeYM,UgxlXMrTyU5Xx2dJzTJ4AaABAg,oiqg,lZOQ4cbN_ps,youtube#comment,rsVZCahUVhCo-PBoV2Q-8RhMb-I,UgxlXMrTyU5Xx2dJzTJ4AaABAg,lZOQ4cbN_ps,Sehr urige Wälder hat Berlin da in einigen Gas...,...,UCTog-0wzpVJ8m-KuDrSVmlQ,True,none,2,2018-04-17T12:23:57Z,2018-04-17T12:23:57Z,True,0,True,NaN
225,youtube#commentThread,-48SXbaX8eUQN382PI9Ih3-Yb_Y,UgxIJcT5_cl6GZnr3cd4AaABAg,rNTM,_tPdhA2EZKg,youtube#comment,tyA9XAYSXXfbx5yWk9POHpv5ZYA,UgxIJcT5_cl6GZnr3cd4AaABAg,_tPdhA2EZKg,thank u this was the last thing i needed cuz i...,...,UCodhAcUsudX2YPCF6Tg1cIw,True,none,0,2021-03-23T11:08:23Z,2021-03-23T11:08:23Z,True,0,True,NaN
1,youtube#commentThread,Oh-NTWh_vFipTc-3BKBTVz0LTEs,UgxLCQCLsca5zhKol2h4AaABAg,VX3C,iHpgw8WN6Bg,youtube#comment,cTsnqv7CcJvgJ0pznTYOoEt7vwY,UgxLCQCLsca5zhKol2h4AaABAg,iHpgw8WN6Bg,<3<3<3<3<3<3\nTrop ADORABLES!,...,UCkHRUDRkrJPUXQfar1hChOg,True,none,0,2012-03-12T14:27:16Z,2012-03-12T14:27:16Z,True,0,True,NaN
246,youtube#commentThread,-oTQomQoxCTaylgj31oCevLH20k,UgzoEv_iAHz3_thddO94AaABAg,MmmP,sg7kIDCCK7o,youtube#comment,up51jWRMOKDyxayrHTqkhnXLhA0,UgzoEv_iAHz3_thddO94AaABAg,sg7kIDCCK7o,MITICO ZOO DI FASANO NEL CUORE DELLA PUGLIA E ...,...,UC6NS9EpC-C-IHbYHYdUqqaA,True,none,7,2011-01-26T14:09:37Z,2011-01-26T14:09:37Z,True,0,True,NaN
287,youtube#commentThread,WwSzISEgo7IusNjHTnWMe0EClGk,UgxfouNNjE4B7kZBI-J4AaABAg,w2KX,Mtc5TKjqgfE,youtube#comment,HYHQmNhp1K5vAcEi1z_kBs8nDbc,UgxfouNNjE4B7kZBI-J4AaABAg,Mtc5TKjqgfE,my favorite video,...,UCYWCCZPE2wUNYKIA4T9VOeA,True,none,0,2008-08-29T21:45:15Z,2008-08-29T21:45:15Z,True,0,True,NaN
